In [53]:
import os
import random
import pandas as pd
from mypy.proj import find_dropbox

In [2]:
cd ../testy/test_02/

C:\proj\src\mozg-rozpajton\testy\test_02


In [93]:
fls = os.listdir()
wzor_file = fls[0]
print(wzor_file)

f = open(wzor_file, encoding='utf-8')
lines = f.readlines()
f.close()

szkic_2.txt


## segment by `{%QUESTION%}`

In [94]:
pattern = '{%QUESTION%}'

in_questions = False
start_text = list()
questions = list()
question_number = 0

for l in lines:
    has_pattern = pattern in l
    if not in_questions: 
        if has_pattern:
            in_questions = True
        else:
            start_text.append(l)
    
    if in_questions:
        if has_pattern:
            questions.append(list())
            question_number = len(questions) - 1
        questions[question_number].append(l)

In [89]:
num_questions = len(questions)
num_questions

4

In [90]:
print(questions[2][1])

Korzystając z funkcji `range` utwórz **listę** wartości od {%WARTOSC_START%} do {%WARTOSC_KONIEC%}.



In [22]:
import re

In [23]:
reg = '{%[A-Z_]+%}'
tekst = 'utwórz **listę** wartości od {%WARTOSC_START%} do {%WARTOSC_KONIEC%}.'

In [27]:
results = re.findall(reg, tekst)
for r in results:
    print(r[2:-2])

WARTOSC_START
WARTOSC_KONIEC


In [39]:
# find all variables
reg = '{%[A-Z_]+[0-9]*%}'
var_names = set()
for l in lines:
    results = re.findall(reg, l)
    for r in results:
        var = r[2:-2]
        if not var in var_names:
            var_names.add(var)

In [40]:
var_names

{'IMIE_NAZWISKO',
 'LISTA',
 'LISTA_VAL',
 'NAPIS',
 'NAPIS_RANGE_1',
 'NAPIS_RANGE_2',
 'NAPIS_RANGE_3',
 'NAPIS_VAL',
 'NAZWA_LISTY',
 'NAZWA_ZMIENNEJ01',
 'QUESTION',
 'WARTOSC_KONIEC',
 'WARTOSC_START',
 'WARTOSC_VAL'}

## general substitution rules

In [42]:
napis = str([[3, 4], ['kot', 'pies']])
print(napis)

[[3, 4], ['kot', 'pies']]


In [44]:
start = [10, 15, 21, 33, 45, 81, 122]
koniec = [5, 14, 28, 39, 57, 73]
napisy = ['od {} do {}'.format(x, x + y) for x in start for y in koniec]

In [95]:
lista_elems = [[3, 4], ['kot', 'pies'], 23, [[8, 7, 6], 'abcd']]

substitute = {'LISTA_VAL': [str(x) for x in [
                            [3, 4], ['pies', 'kot'], ['kot', 'pies'],
                            [8, 7, 6], [8, 6], [7, 6]]],
              'NAPIS_VAL':  ['pie', 'ot', 'dcba', 'bd' 'seip', 'tok'],
              'WARTOSC_VAL': [3, 4, 23, 8, 7, 6],
              'NAZWA_LISTY': ['jamnik', 'papier', 'kubrak', 'szyk', 'krzyk', 'koszyk'],
              'NAZWA_ZMIENNEJ01': ['krab_szczepan', 'jamnik_dobromir', 'sarna_stefania',
                                    'wilk_wojtek', 'owca_agnieszka'],
              'NAPIS':      ['super kartkówka!', 'uwielbiam kartkówki!',
                             'python prowadzi mnie; przez analizy eeg', 'python skradł mi serce',
                             'zawsze dokładnie przygotowuję się do zajęć',
                             'zawsze przychodzę na zajęcia punktualnie'],
              'WARTOSC_START': [1, 2, 3, 4, 5, 6, 7, 8],
              'WARTOSC_KONIEC': [3, 4, 5, 6, 7],
              'NAPIS_RANGE_1': napisy,
              'NAPIS_RANGE_2': napisy,
              'NAPIS_RANGE_3': napisy}

## read students list

In [49]:
dropbox_dir = find_dropbox()
lista_path = os.path.join(dropbox_dir, 'Sarenka', 'ZAJECIA', 'python eeg warsztaty 2016')
lista_name = os.listdir(lista_path)[0]

print(lista_path)
print(lista_name)

C:\Users\swps\Dropbox\Sarenka\ZAJECIA\python eeg warsztaty 2016
lista_studentow.txt


In [54]:
lista = pd.read_table(os.path.join(lista_path, lista_name), sep='\t')

im_nazw = lista['Nazwisko i imię'].tolist()
short_im = [(name.split()[-1][0] + name.split()[0]).lower() for name in im_nazw]

def depol(txt):
    pol2ascii = {'ż': 'z', 'ó': 'o', 'ź': 'z', 'ć': 'c',
                 'ę': 'e', 'ą': 'a', 'ł': 'l', 'ś': 's'}
    return ''.join([pol2ascii.get(ch, ch) for ch in txt])

short_im = list(map(depol, short_im))
print(short_im[5:10])

['mgogolewska', 'eisakova', 'wkowalczyk', 'mkowalska', 'zkucharek']


## test deepcopy + shuffle

In [79]:
lst = deepcopy(lista_elems)
random.shuffle(lst)
print(lst)
print(lista_elems)

[[3, 4], [[8, 7, 6], 'abcd'], ['kot', 'pies'], 23]
[[3, 4], ['kot', 'pies'], 23, [[8, 7, 6], 'abcd']]


everything works well

## generate tests

In [67]:
def subst(text, substitutions):
    for k in substitutions.keys():
        realkey = '{%' + str(k) + '%}'
        if realkey in text:
            text = text.replace(realkey, str(substitutions[k]))
    return text

In [96]:
for im, imshrt in zip(im_nazw, short_im):

    # randomize questions order
    question_order = list(range(num_questions))
    random.shuffle(question_order)
    this_questions = [questions[i].copy() for i in question_order]
    
    # generate current substitutions:
    this_substitution = substitute.copy()
    for k in this_substitution.keys():
        this_substitution[k] = random.choice(this_substitution[k])
    
    # additional things for subst:
    this_substitution['WARTOSC_KONIEC'] += this_substitution['WARTOSC_START']
    
    lista = deepcopy(lista_elems)
    random.shuffle(lista)
    this_substitution['LISTA'] = lista

    # generate substituted questions:
    for qi, q in enumerate(this_questions, 1):
        for i in range(len(q)):
            q[i] = subst(q[i], this_substitution)
            if '{%QUESTION%}' in q[i]:
                q[i] = q[i].replace('{%QUESTION%}', 'ZADANIE {}  \n'.format(qi))
    
    # generate substituted start text:
    this_start_text = start_text.copy()
    for i in range(len(this_start_text)):
        if '{%IMIE_NAZWISKO%}' in this_start_text[i]:
            this_start_text[i] = this_start_text[i].replace('{%IMIE_NAZWISKO%}', im)
    
    # write test to disk:
    fname = imshrt + '.md'
    f = open(fname, mode='w', encoding='utf-8')
    
    f.writelines(this_start_text)
    for q in this_questions:
        f.writelines(q)
    
    f.close()

In [60]:
# ups, chcieliśmy faktycznie markdown:
fls = os.listdir()
fls[:3]

['abarszczewska.txt', 'adontsova.txt', 'aruban.txt']

In [61]:
import shutil

In [62]:
for f in fls:
    if not f == 'wzor.txt':
        shutil.copyfile(f, f.replace('.txt', '.md'))

In [63]:
# eh, should have used `shutil.move()`...

## problemy z poprawnym kodowaniem pliku

unresolved:
* `windows-1250` vs `utf-8` encoding...

In [64]:
f = open('kmajcher.md')

In [65]:
f.encoding

'cp1250'

In [66]:
text = f.readlines()
print(text[:7])

['KartkĂłwka dla:\n', 'Majcher Katarzyna\n', '\n', 'ZADANIE 1\n', 'Masz zmiennÄ… `kałamarnica`, ale za cholerÄ™ nie wiesz jakiego rodzaju to zmienna (np. `str` vs `int` itp.) - napisz poniĹĽej komendÄ™, ktĂłra Ci w tym pomoĹĽe:\n', '```\n', '\n']


In [68]:
f.close()
f = open('kmajcher.md', encoding='utf-8', errors='ignore')
text = f.readlines()
f.close()
print(text[:7])

['Kartkówka dla:\n', 'Majcher Katarzyna\n', '\n', 'ZADANIE 1\n', 'Masz zmienną `kaamarnica`, ale za cholerę nie wiesz jakiego rodzaju to zmienna (np. `str` vs `int` itp.) - napisz poniżej komendę, która Ci w tym pomoże:\n', '```\n', '\n']


To jest ten zły character
�

In [75]:
# let's just omit bad char...
subdir = 'fin_tests'
this_dir = os.getcwd()
fls = os.listdir(this_dir)

if not os.path.exists(os.path.join(this_dir, subdir)):
    os.mkdir(subdir)

for file in fls:
    if not file in ['wzor.txt', subdir]:
        f = open(file, encoding='utf-8', errors='ignore')
        text = f.readlines()
        f.close()
        
        # save
        f = open(os.path.join(subdir, file), mode='w', encoding='utf-8')
        f.writelines(text)
        f.close()